In [1]:
import xarray as xr
import proplot as pplt
import numpy as np
import os
import s3fs
from matplotlib import pyplot as plt
from mats_l1b_tools.geolocation import addTPdata
from mats_l1b_tools.fetch_data import fetch_MATS_l1b_data

ModuleNotFoundError: No module named 'proplot'

In [2]:
# Reads data (diurnal / semidiurnal) and plots the sum of all tidal components for u, v, w, T and rho
# Control which components by changing the lists

datadir = '/Users/lindamegner/MATS/MATS-retrieval/data/CMTM5541913/'
outputdir = '/Users/lindamegner/MATS/MATS-retrieval/MATS-analysis/Linda/output'

fig = pplt.figure(figwidth='18.3cm')
array = [  # the "picture" (0 == nothing, 1 == subplot A, 2 == subplot B, etc.)
    [1, 1, 2, 2],
    [3, 3, 4, 4],
    [0, 5, 5, 0],
]

ax = fig.subplots(array,proj='cyl')


ax.format(coast=True,landzorder=4,latlines=30, lonlines=60,abc=True, abcloc='ul')

onewave="e1"
diurnal=True
variables=[f"{wave}_u",f"{wave}_v",f"{wave}_w",f"{wave}_t",f"{wave}_d"]
variables=["_u","_v","_w","_t","_d"]
variables_str=['u','v','w','T','rho']
diurnal_waves=['w1','w2','w3','w4','e1','e2','e3','s0']
diurnal_waves=['e1'] # example
#diurnal_waves=['s0'] # example
urnal = ['diurnal','semidiurnal']
#urnal = ['semidiurnal']

LST=17.5 # in hours
altitude=87.5 # in km
mon=2 # month

i = 0
datax = xr.Dataset()
for variable in variables:
    first=True

    for urnal_str in urnal:

        for wave in diurnal_waves:

            #if (wave == 'w1' and urnal_str == 'diurnal') or (wave == 'w2' and urnal_str == 'semidiurnal'):
                #print(f'SKIPPED! {wave} and {urnal_str}')
            #    b=5

            #else:

            lon=np.linspace(-np.pi,np.pi,360)

            diurnal = xr.open_dataset(f'{datadir}ctmt_{urnal_str}_2002_2008.nc')
            diurnal = diurnal.sel(lev=altitude)
            diurnal = diurnal.sel(month=mon)

            phasevar=f"phase_{wave}{variable}"
            ampvar=f"amp_{wave}{variable}"

            try:

                amp = diurnal[ampvar]
                units = amp.units
                phase = diurnal[phasevar]

                amp = np.resize(diurnal[ampvar],(len(lon),len(diurnal[ampvar])))
                phase = np.resize(diurnal[phasevar],(len(lon),len(diurnal[phasevar])))
                lon = np.resize(lon, (len(diurnal[phasevar]),len(lon)))

                if wave[0] == "w":
                    s = - int(wave[1])
                    #print(f's = {s}')
                if wave[0] == "e":
                    s = int(wave[1])
                    #print(f's = {s}')
                
                if wave[0] == "s":
                    s = 0
                    #print(f's = {s}')

                if urnal_str == 'diurnal':
                    n = 1

                else:
                    n = 2

                if first:
                    data=amp.T*np.cos(n*2*np.pi/24*(LST-phase.T)-(s+n)*lon)
                    first=False
                else:
                    #print(wave)
                    data=data+amp.T*np.cos(n*2*np.pi/24*(LST-phase.T)-(s+n)*lon)


            except:
                #print(f'No {wave} in {urnal_str}')
                b=5

    datax[variable] = (('lat','lon'), data)
    datax[variable].attrs["units"] = units

    m=ax[i].contourf(lon[0,:]/np.pi*180,diurnal.lat,data,robust=98,extend='both',title=variable)
    ax[i].colorbar(m,loc='b',label=units)
    ax[i].format(title=variables_str[i])
    i = i + 1

datax['lon'] = lon[0,:]/np.pi*180
datax['lat'] =  diurnal.lat

fig.suptitle(f'Tidal fields at {altitude} km - February - LST {LST}')


fig.savefig(f'{outputdir}{altitude}alt{altitude}_month{mon}_LST{LST}.png',format='png')


NameError: name 'pplt' is not defined

In [ ]:
# This script calculates the phase term of atmospheric tides for a range of zonal wavenumbers (s)
# and harmonics (n), using the formula: phase = n * Ω * tUT + s * λ - Φs,n
Omega = 360 # 2 * np.pi  # Earth's rotation rate in radians per day
lambda_rad = np.radians(45)  # Example longitude in radians (e.g., 45 degrees east)
tUT = 0.5  # Example Universal Time in days (e.g., noon)
Phi_s_n=0

if wave[0] == "w":
        s = - int(wave[1])
        #print(f's = {s}')
elif wave[0] == "e":
        s = int(wave[1])
        #print(f's = {s}')
elif wave[0] == "s":
        s = 0
else:
        s=1
        print('s is set to default value of 1')
        
if diurnal:
        n=1
else:
        print('n is sen to default value of 2')

        
# Calculate the phase term
phase = n * Omega * tUT + s * lambda_rad - Phi_s_n
print('phase:',phase)



#%% Fetches info for MATS data
ds = fetch_MATS_l1b_data('IR2')

# %%
# Plot the 1000th image using Xarrays built in plotting fuction
ds.ImageCalibrated.isel(time=1000).plot()


#%%
# Plot the 1000th image using matplotlib
plt.figure()
plt.imshow(ds.ImageCalibrated.isel(time=1000).values,origin='lower',aspect='auto')
plt.xlabel('Image Column Number')
plt.ylabel('Image Row Number')
plt.colorbar()
plt.title('Image from IR2')
plt.show()

phase: 3.9269908169872414
